In [ ]:
%load_ext autoreload
%autoreload 2

### IMPORTING FILES FROM THE PROJECT ROOT.

In [ ]:
import time
import numpy as np
import random
from matplotlib.markers import MarkerStyle
from matplotlib.lines import Line2D
from matplotlib import pyplot as plt


#### --- IMPORTING FROM LOCAL FILES ---
import SortingMeasurement
import AlgoritmiDiOrdinamento

#### --- MAKING ROOT PROJECT FOLDER VISIBLE AT SCRIPT LEVEL ---

##
## --- Finding root project folder ---
##     (needed to import settings)
##
import os, sys

def find_project_root(beacon="ROOT_BEACON"):
    current_path = os.getcwd()
    
    while True:
        if os.path.isfile(os.path.join(current_path, beacon)):
            return current_path
        
        parent_path = os.path.dirname(current_path)
        if parent_path == current_path:
            raise RuntimeError(f"Unable to find project root.\nNo '{beacon}' file found in any parent directory.")
        
        current_path = parent_path

# --- Find and set project root ---
project_root = find_project_root()

## --- Adding root porject folder to current system path ---
os.chdir(project_root)
    
####                    --- end ---

#### IMPORTING FROM ROOT PROJECT FOLDER 
from Utils import ArrayStorageCompressor, ArrayDataManager, ArraySettings, SortingSettings

### LOADING DATA FROM ARRAY FOLDER

In [ ]:
%%time

storage_array_files = ArraySettings.GET_COMPRESSED_ARRAY_FILES_IN_STORAGE_FOLDER(get_all= True)
imported_arrays = ArrayDataManager.FoldersDataStorage()



for storage_array_file_path in storage_array_files:
    raw_data = ArrayStorageCompressor.readFromFile(
        path= storage_array_file_path,
        return_file_name= False,
        decompress= False
    )
    storage_name = os.path.dirname(storage_array_file_path)
    imported_arrays.update(folder_path= storage_name, array_sample_container= raw_data)

### CALCULATE EXECUTION TIMES

##### CREATE SUBPROCESSES TO INDIVIDUALLY CALCULATE EXECUTION TIMES

In [ ]:
%%time
##                 ->#     CHANGE THIS FUNCTION     #<-
algorithms = [AlgoritmiDiOrdinamento.CountingSort, AlgoritmiDiOrdinamento.QuickSort, AlgoritmiDiOrdinamento.RadixSort, AlgoritmiDiOrdinamento.QuickSort3Way]
arrayTimeManager = ArrayDataManager.ExecutionTimeDataStorage()

sorting_algorithm = SortingMeasurement.MeasurableTimeExecutionAlgorithm() 

for algorithm in algorithms:
    sorting_algorithm.set(algorithm)
    assert sorting_algorithm.get() != None, f"No sorting algorithm set, please use: SortingMeasurement.SortingAlgorithm.set('<# ALGORITHM FUNCTION #>')"
    for folder_path, array_sample_container in imported_arrays.items():
        arrayTimeManager.update(
            algorithm = sorting_algorithm.get_name(),
            array_folder = folder_path,
            execution_times = SortingMeasurement.measure_container_array(
                array_sample_container= array_sample_container,
                function= sorting_algorithm.execute,
                minTime= None
            )
        )

print("Execution times calculated!")

### CALCULATE PLOT POINTS USING CURRENT EXECUTION TIMES

### PLOT EXECUTION POINTS

In [ ]:
def show_time_results():
    fig = plt.figure()
    ax = fig.add_subplot()
    
    chunk_times = []
    for algorithm, measurement_storage in arrayTimeManager.items():
        marker_style = list(MarkerStyle.markers.keys())[random.choice([i for i in range(len(MarkerStyle.markers)-4)])]
        line_style = str(list(Line2D.lineStyles.keys())[random.choice([i for i in range(len(Line2D.lineStyles)-3)])])
        for i, folder_path in enumerate(measurement_storage.keys()):
            x, y, e = [], [], []
            last_variability = (-1, -1)
            time_data = arrayTimeManager.get_execution_times(algorithm, folder_path)
            for dict_index, array_execution in enumerate(time_data):
                array_execution.compute_time_analysis()
                time_analysis = array_execution.time_analysis
                #assert last_variability[1] < metadata_dict.get_metadata()["variability"], f"Order problem: {(last_variability, metadata_dict.get_metadata()['variability'])}. Dict id: {dict_index}"
                last_variability = (dict_index, array_execution.get_variability())
                x.append(array_execution.get_variability())
                y.append(time_analysis.mean)
                e.append(time_analysis.sd)
            chunk_times.append(sum(y))
            line = ax.errorbar(x, y, yerr=e, linestyle= line_style, ecolor = "black", marker=marker_style)
            line.set_label(algorithm + folder_path[folder_path.rfind("\\"):])
    fig.suptitle("Execution time", fontsize=14, fontweight="bold")
    ax.set_xlabel("Array Length" if ArraySettings.VARIABILITY == ArraySettings.Variability.onLength else "Different Numbers")
    ax.set_ylabel("Execution Time (s)")
    ax.legend()
    #ax.errorbar(length_values, time_values, yerr=error_values, fmt="r--o", ecolor = "black")
    plt.show()

In [ ]:
show_time_results()

### SAVING ARRAY MANAGER OBJECT LOCALLY

In [ ]:
destinationFolder = os.sep.join([project_root, SortingSettings.EXECUTION_TIMES_FOLDER])
os.makedirs(destinationFolder, exist_ok= True)
for algorithm in arrayTimeManager.keys():
    destination_file = algorithm + SortingSettings.EXECUTION_TIMES_FILE_EXTENSION
    destination_file_path = os.sep.join([destinationFolder, destination_file])
    if os.path.exists(destination_file_path):
        algorithm_arrayTimeManager = ArrayStorageCompressor.readFromFile(destination_file_path, decompress= False)
        algorithm_arrayTimeManager.merge({algorithm: arrayTimeManager[algorithm]})
        print(f"Updating existing file at {destination_file_path}")
        data_to_save = algorithm_arrayTimeManager
    else:
        data_to_save = ArrayDataManager.ExecutionTimeDataStorage({algorithm: arrayTimeManager[algorithm]})
    
    ArrayStorageCompressor.writeOnFile(data_to_save, destination_file_path)
    print(f"Execution timing data saved at {destination_file_path}")